テストセットは600個
トレーニングセットが800個（変更なし）
タイトルを日本語にして、置くこと（調べてやっておく）

乱数の種を変えること。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal,permutation
import pandas as pd
from pandas import DataFrame, Series 
import matplotlib as mpl
mpl.rcParams['font.family'] = 'IPAMincho'

C:\Users\Makuwo\Anaconda3\envs\makuwo\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%config IPCompleter.greedy=True
%matplotlib inline
%matplotlib notebook

In [3]:
# plt.matplotlib_fname()

In [4]:
np.random.seed(19961123)

n0, mu0, variance0 = 800, [10,11], 20
data0 = multivariate_normal(mu0,np.eye(2)*variance0, n0)
df0 = DataFrame(data0,columns=['x','y'])
df0['t'] = 0

n1, mu1, variance1 = 600, [18,20], 22
data1 = multivariate_normal(mu1,np.eye(2)*variance1, n1)
df1 = DataFrame(data1,columns=['x','y'])
df1['t'] = 1

df = pd.concat([df0,df1], ignore_index=True)
df = df.reindex(permutation(df.index)).reset_index(drop=True)

num_data = int(len(df)*0.8)
train_set =df[:num_data]
test_set =df[num_data:]

In [5]:
train_x=train_set[['x','y']].as_matrix()
train_t=train_set['t'].as_matrix().reshape([len(train_set),1])
test_x=test_set[['x','y']].as_matrix()
test_t=test_set['t'].as_matrix().reshape([len(test_set),1])

In [6]:
x = tf.placeholder(tf.float32,[None ,2])
w = tf.Variable(tf.zeros([2,1]))
w0 = tf.Variable(tf.zeros([1]))
f = tf.matmul(x,w) + w0
p = tf.sigmoid(f)

In [7]:
t = tf.placeholder(tf.float32, [None,1])
loss = -tf.reduce_sum(t*tf.log(p)+(1-t)*tf.log(1-p))#定義式注意
train_step = tf.train.AdamOptimizer().minimize(loss)

In [8]:
correct_prediction = tf.equal(tf.sign(p-0.5),tf.sign(t-0.5))
accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [10]:
train_accuracy = []
test_accuracy = []
for _ in range (2500):
    sess.run(train_step,feed_dict={x:train_x,t:train_t})
    acc_val =sess.run(accuracy, feed_dict={x:train_x,t:train_t})
    train_accuracy.append(acc_val)
    acc_val =sess.run(accuracy, feed_dict={x:test_x,t:test_t})
    test_accuracy.append(acc_val)

In [ ]:
i = 0
train_y = []
for _ in range (200000):
    i += 1 
    sess.run(train_step,feed_dict={x:train_x,t:train_t})
    loss_val =sess.run(loss, feed_dict={x:train_x,t:train_t})
#     train_y.append(loss_val)
    if i % 1000 == 0:
        loss_val =sess.run(loss, feed_dict={x:train_x,t:train_t})
        train_y.append(loss_val)
#         time.append(i)
        print ('Step:%d, Loss:%f'%(i, loss_val))


Step:1000, Loss:441.786377
Step:2000, Loss:396.281433
Step:3000, Loss:361.117889
Step:4000, Loss:333.711578
Step:5000, Loss:312.292542
Step:6000, Loss:295.591431
Step:7000, Loss:282.662262
Step:8000, Loss:272.775238
Step:9000, Loss:265.359924
Step:10000, Loss:259.953705
Step:11000, Loss:256.174286
Step:12000, Loss:253.695694
Step:13000, Loss:252.225616
Step:14000, Loss:251.486877
Step:15000, Loss:251.209305
Step:16000, Loss:251.147415
Step:17000, Loss:251.141968
Step:18000, Loss:251.141876
Step:19000, Loss:251.141876


In [ ]:
loss_processing = []
c = 0
for c in range(2500):
        loss_processing [c] = train_y[c] / 280

In [ ]:
fig_R = plt.figure(figsize=(8,6))
subplot = fig_R.add_subplot(1,1,1)
subplot.plot(range (len(train_accuracy)),train_accuracy,linewidth=2,label='Training set')
subplot.plot(range (len(test_accuracy)),test_accuracy,linewidth=2,label='Test set')
subplot.legend(loc='upper left')

fig_L = plt.figure(figsize=(8,6))
subplot = fig_L.add_subplot(1,1,1)
subplot.plot(train_y,train_y,linewidth=2)
# subplot.legend(loc='upper left')
plt.title("ロジスティク回帰による二項分類器")